# Model tutorial
Michael Chimento
## Table of Contents

* [Introduction](#introduction)
* [Behaviors](#behaviors)
* [Agents](#agents)
    * [Instance variables: knowledge state, memory and parameter values](#agents_1)
    * [Class methods: observation and memory](#agents_2)
    * [Class methods: EWA functions](#agents_3)
* [NBDA](#NBDA)
* [Networks](#networks)
* [Simulation](#simulation)
    * [Helper functions](#helper-agent)
    * [Helper functions: turnover](#helper-turnover)
    * [Helper functions: extracting data](#helper-extract)
* [Running the simulation](#running)
    * [Simulation parameters](#sim_params)
    * [Population parameters](#pop_params)
    * [EWA parameters](#EWA_params)
    * [NBDA parameters](#NBDA_params)
    * [Execution](#run_sim)
    * [Data management](#data_mgmt)

## Introduction <a class="anchor" id="introduction"></a>

This companion document is meant to show how we have implemented our agent based model in Python3. We used an object oriented programming style to increase the flexibility of the model, and make life easier when the model becomes complicated or is extended.

First we load the required libraries.

In [ ]:
#!/usr/bin/env python

from collections import Counter
import math
import sys
import networkx as nx
import random
import numpy as np
from os.path import isfile,join
import pandas as pd
from os import listdir,remove
import time
from scipy.special import logsumexp
import jdc

## Behaviors <a class="anchor" id="behaviors"></a>

Let's first define a class called `behavior`. Each instance of `behavior` will represent a possible behavior that agents will be able to learn during the simulation. Any python class can be given its own variables and methods using `self.var_name` convention. Here, we define a initialization method that assigns each instance of a behavior with a name, an associated payoff, and a base learning rate. The payoff defines the reward obtained for producing the behavior, and the base rate is used in the NBDA equation that determines whether an agent will acquire the behaviour at any given timestep. The base hazard represents the likelihood of acquiring the behavior at each timestep which is independent of any social or asocial influence. A higher base base value means that the behavior will be easier to acquire.

In [ ]:
#define the class behavior
class behavior:
    def __init__(self,name,payoff,base_rate):
        self.name = name #string name of behavior
        self.payoff = payoff #float payoff of behavior
        self.base_rate = base_rate #baseline learning parameter (lambda_0) from NBDA

## Agents <a class="anchor" id="agents"></a>

Next, let's define the `agent` class. During a simulation, each node in our network will be occupied by an instance of this class.

### Instance variables: knowledge state, memory and parameter values <a class="anchor" id="agents_1"></a>
We will initialize agents with a few different instance variables, including:

* `self.id` a unique ID, 
* `self.knowledge` a dictionary that will hold the behaviors that the agent knows how to produce. Keys are the behavior names, and values are the associated information with each behavior, including its attraction score, individual and social weights for behavior, the final probability of producing the behavior.
* `self.ind_memory` a list that will hold a history of behaviors that they've produced themselves within the memory window
* `self.temp_social_memory` a list that is a short-term social memory, which holds all the behaviors that an agent has observed its neighbors produce in the current time step
* `self.long_social_memory` a list that is a long-term social memory, which holds all the behaviors that an agent has observed its neighbors produce within the memory window.
* `self.exposure` represents how many timesteps the agent has been in the simulation, or exposed to the simulation environment. This will be later used for informing a population turnover function.
* `self.naive` a boolean value that represents whether an agent knows any behaviors at all

Each agent is also initialized with some parameter values that will affect it's behavioral productions. These parameter values can vary from agent to agent, and can even be redefined using a function, all depending on the requirements of the simulation. This allows for flexible definitions of population heterogeneity. 
* `self.sigma` the agent's social information bias parameter (used in EWA)
* `self.phi` the agent's recent information bias parameter (used in EWA)
* `self.f_SI` the agent's conformity exponent (used in EWA)
* `self.tau` the agent's sensitivity to differences in attraction scores (used in EWA)

In [ ]:
class agent:
    def __init__(self,ID):
        self.id = ID #keeps agents identifiable across simulations
        self.knowledge = {} #name, a_mat, i_mat, s_mat, p_mat,solve_count
        self.ind_memory = [] #records agent's own productions for the duration of the memory window
        self.temp_social_memory = [] #records observed productions for the duration of the timestep
        self.long_social_memory = [] #records lists of lists, each sublist from a single timestep within the memory window
        self.naive = True # binary variable that flips to False once any behavior has been learned by the agent

        #EWA parameters
        self.sigma = sigma #social information bias
        self.phi = phi #recent payoff bias
        self.f_SI = f_SI #conformity of social influence exponent
        self.tau = tau #inverse exploration parameter (conservatism)


### Class methods: observation and memory <a class="anchor" id="agents_2"></a>

The following class methods handle aspects of the agents memory of their own productions, and their social productions.

* `observe(behavior)` is called after each behavior is produced. An agent may only observe it's neighbors, and the observed behavior is added to its short term social memory.
* `consolidate_social_memory()` is called once per timestep, after all agents have produced their behaviors. It adds the memory from the current timestep to the long term social memory.
* `prune_ind_memory(timestep)` is called once per timestep and, using a first in, first out (FIFO) rule, removes any behaviors from an agent's individual memory that were produced outside of the memory window.
* `prune_social_memory(timestep)` is called once per timestep and, using a FIFO rule, removes any behaviors from an agent's social memory that were produced outside of the memory window.
* `reset_temp_social_memory()` erases all observed behaviours from the previous timestep in preparation for the next set of observations.


In [ ]:
%%add_to agent

def observe(self,behavior):
        if behavior in self.knowledge:
            self.temp_social_memory.append(behavior)
            #print("observation by {}: {}".format(self.id, self.knowledge[behavior]))

def prune_ind_memory(self):
    while len(self.ind_memory) > memory_window:
        self.ind_memory.pop(0)
    #assert len(self.ind_memory) <= memory_window, print("Individual memory has exceeded memory window.",len(self.ind_memory))

def prune_social_memory(self):
    while len(self.long_social_memory) > memory_window:
        self.long_social_memory.pop(0)
    #assert len(self.long_social_memory) <= memory_window, print("Social memory has exceeded memory window.",len(self.long_social_memory))

def consolidate_social_memory(self):
    self.long_social_memory.append(self.temp_social_memory)

def reset_temp_social_memory(self):
    self.temp_social_memory = []


### Class methods: EWA functions <a class="anchor" id="agents_3"></a>

Each agent contains a method that calculate an attraction score from received behavioral payoffs. This attraction score is turned into an individual weight using a softmax function. The agents also keep track of what behaviors their neighbors produce within the memory window. These values are turned into a social weight. Finally, the individual and social weights are combined into a final probability that the agents would produce a given behavior. This is summarized in the steps below.

`A_mat_update(behavior)` updates an agent's attraction score matrix using the following equation: $A_{kt} = \phi\pi_k + (1-\phi)A_{k,t-1}$

In [ ]:
%%add_to agent
def A_mat_update(self, produced_behavior):
    reward = all_behaviors[produced_behavior].payoff
    for known_behavior in self.knowledge.keys():
        if known_behavior==produced_behavior:
            new_A_kt = (1 - self.phi)*self.knowledge[known_behavior]["a_mat"] + self.phi*reward
        else:
            new_A_kt = (1 - self.phi)*self.knowledge[known_behavior]["a_mat"] #null payoff for non-produced behavior
        self.knowledge[known_behavior]["a_mat"] = new_A_kt

`I_mat_update(behavior)` updates an agent's individual weight matrix using the following equation: $I_{kt} = \frac{exp(\tau A_{kt})}{\sum_k^Z{exp(\tau A_{kt})}}$. The use of the softmax equation here guarantees that the sum of all values for known behaviors is 1. To avoid overflow or underflow errors, we have calculated $I_kt$ in log space, and will exponentiate it in `P_mat_update`.

In [ ]:
%%add_to agent
def I_mat_update(self):
    if not self.naive:
        A_mat = np.array([behavior["a_mat"] for behavior in self.knowledge.values()])
        tau_by_A_mat = np.multiply(A_mat, self.tau)
        summed_A_mat = logsumexp(tau_by_A_mat)
        for count, behavior in enumerate(self.knowledge.values()):
            behavior["i_mat"] = tau_by_A_mat[count] - summed_A_mat

`S_mat_update(behavior)` updates an agent's social weight matrix using the following equation: $S_{kt} = \frac{(\sum_{t}^{m}{n_{kt}})^{f_{SI}}}{\sum_k^{Z}(\sum_t^m{n_{kt}})^{f_{SI}}}$. This equation ensures that the values for each known behavior are bounded between 0 and 1. First the long term memory is flattened. $n$ values are determined using the `count()` function on the long term memory list. The denominator is calculated first to save later computation, and then each new social weight matrix value is calculated for known behaviors, and written into the knowledge dictionary under that behavior key's `["s_mat"]` value.

In [ ]:
%%add_to agent
def S_mat_update(self):
    if not self.naive:
        social_memory = [item for subl in self.long_social_memory for item in subl]
        assert len(social_memory) <= (N * memory_window), print("memory {} exceeds memory window {}".format(len(social_memory),N*memory_window))
        if len(social_memory)>0:
            denom=0
            for behavior in self.knowledge.keys():
                denom += social_memory.count(behavior)**self.f_SI
            for behavior in self.knowledge.keys():
                new_S_kt = (social_memory.count(behavior)**self.f_SI) / denom
                self.knowledge[behavior]["s_mat"] = new_S_kt

        #if no social memory present
        else:
            for count,behavior in enumerate(self.knowledge.values()):
                new_S_kt = 0
                behavior["s_mat"] = new_S_kt

`P_mat_update(behavior)` updates an agent's probability matrix using the following equation: $P_{kt} = (1-\sigma)I_{kt} + \sigma S_{kt}$. If an agent has no experience observing others, the value from their individual weight matrix is the final production probability.

In [ ]:
%%add_to agent
def P_mat_update(self):
    if not self.naive:
        social_memory = np.array([x["s_mat"] for x in self.knowledge.values()])
        if np.sum(social_memory)>0:
            for behavior in self.knowledge.values():
                behavior["p_mat"] = (1 - self.sigma)*np.exp(behavior["i_mat"]) + self.sigma*behavior["s_mat"]

        else:
            for behavior in self.knowledge.values():
                behavior["p_mat"] = np.exp(behavior["i_mat"])

Finally, we need the agents to do something with the knowledge they have. They can produce behaviors using the following method. This creates a list of possible behaviors from their knowledge dictionary keys. One behavior is chosen from using Numpy's `random.choice()` function, with the probability values passed as the probability argument to weight the choice.

That behavior is then added to the agent's individual memory, and is passed to `A_mat_update()` to update attraction, then individual weight, and finally probability (in case the simulation allows for more than one behavioral production per timestep). The name of the behavior produced is returned.

In [ ]:
%%add_to agent
def produceBehavior(self):
    behavior_choices = [behavior for behavior in self.knowledge.keys()]
    p_mat = np.array([x["p_mat"] for x in self.knowledge.values()])


    #choose production with probability from p_mat
    try:
        production = np.random.choice(behavior_choices, p=p_mat)
    except Exception as e:
        print("Error producing behavior:", e)
        print(sum(p_mat))


    self.ind_memory.append(production) #adds production to memory

    self.A_mat_update(production) #update attraction score for next production

    return production

The final method definition in the agent class is `acquire_behavior(G)` which takes the social network object G as an argument, is run once per timestep, and determines whether agents learn a behavior in a given timestep. First, it creates a list of neighboring nodes of the focal agent. Then, for every behavior that's not known to the focal agent, it calculates the probability of acquisition. This requires the use of the NBDA equation, which takes into account information about the agent's neighbors. This will be covered in the next section. Then, a random number between $[0,1)$ is generated, and if this number is smaller than the acquisition probability, the focal agent acquires the behavior. 

In [ ]:
%%add_to agent
def acquire_behavior(self, G):
    neighbors = [node for node in G.neighbors(self.id)]
    for behavior.name in all_behaviors:
        if behavior.name not in self.knowledge.keys():
            acquisition_prob = lambda_t(behavior.name,neighbors,G)
            assert 0 <= acquisition_prob <=1, "resulting acquision prob from NBDA must be between 0 and 1"
            if random.random() <= acquisition_prob:
                self.knowledge[behavior.name] = {"a_mat": 0,"i_mat": 0,"s_mat":0,"p_mat":0}
                self.I_mat_update()
                self.S_mat_update()
                self.P_mat_update()
                self.naive=False

## NBDA <a class="anchor" id="NBDA"></a>

While the production probabilities are handled by methods in the `agent` class, the calculation of acquisition probabilities by NBDA equations is handled by the following modular series of functions. The basic form of NBDA is given by $\lambda_i(t) = \lambda_0(t)(T(a_i,z(t))+A)(1-z_i(t))$.

* $\lambda_{ik}(t)$ - the rate of transmission for individual $i$ at time $t$ for behavior $k$.
* $ \lambda_{0k}(t)$ - the baseline rate of transmission at time $t$ for behavior $k$
* $(T(a_i,z_k(t))) = s\sum{a_{ij}z_{jk}(t)}$ - the transmission function, which can be represented by a variety of forms accommodating simple or complex contagion. This is composed of:
 + $s$ - the rate of social acquisition per unit connection, and can take values from $[0,\infty]$
 + $a_i$ - the association matrix for the focal individual representing all connections to neighbors
 + $z_jk(t) = \frac{n_{kjt}}{\sum{n_{kjt}}}$ - In the FSSL model, this takes a value between 0 or 1, such that it includes information about the production frequency of the behavior within the memory window. NB: in traditional NBDA, this is a binary variable, representing the knowledge state of the neighbor.
* $A$ - is the presence of asocial learning, in this version of the model it's binary, but could be further defined by ILVs
* $(1-z_{ik}(t))$ ensures the equation equals zero if the behavior is already known.

We will cover how this is implemented in detail below.

First, we have a convenience function that is called within the simulation, once per timestep, that loops through all agents and calls the `acquire_behavior()` method.


In the preceeding code section for the agent method `acquire_behavior()`, `acquisition_prob` requires a call to `lambda_t()`. This function is at the "top" level of NBDA, and first calculates the value for $T(a_i,z_k(t)))$, $A$, and with those then $\lambda_i(t)$. That value is a rate, which is converted into a probability using $P(acquire)=1-exp(-\lambda_i(t))$. This probability is returned for further use in `acquire_behavior()`.

In [ ]:
def lambda_t(behavior, neighbors, G):
    transmission_func = transmission_function(behavior, neighbors, G)

    asocial_learning_func = A_param()

    #lambda(t) = baseline rate function*(transmission_func + ind_learning_parameters)
    acq_rate = all_behaviors[behavior].base_rate * (transmission_func + asocial_learning_func)

    #convert from rate to probability within the last timestep that an agent has acquired the behavior
    acq_prob = 1-math.exp( -acq_rate )

    return acq_prob

The `transmission_function()` takes information about the knowledge states of neighbors and multiplies it by the $s$ parameter, which represents the strength of social learning. Here, we have implemented several forms which this function could take, whose definitions are taken from (Firth et al., 2020).

* `NBDA_type=0` the "default" NBDA, in which $s$ is unbounded and $z_{jk}(t)$ is passed as its raw form.
* `NBDA_type=1` NBDA in which $s$ and $A$ are linked ($A=1-s$) and bounded between $[0,1]$.
* `NBDA_type=2` "proportional rule" NBDA, $z_{jk}(t)$ is normalized by the total number of neighbors.
* `NBDA_type=3` "frequency dependent rule" NBDA, $z_{jk}(t)$ is exponentiated by a conformity exponent, in the same manner as in the EWA equations.

In [ ]:
def transmission_function(behavior,neighbors,G):
    
    z_jt_param = z_jt(behavior, neighbors, G)
    
    #Hoppitt_Laland 2013 General form, s is unbounded
    #lambda(t) = baseline rate function*(rate of acquisition * sum(a_ij*z_j(t)) + A
    if NBDA_type == 0 or NBDA_type == 1:
        transmission_func = s_param * z_jt_param

    #Proportional rule model Firth 2020
    elif NBDA_type == 2:
        assert z_jt_type == "binary", "z_jt must be binary to appropriately calculate NBDA type 2 (proportional rule)"
        transmission_func = s_param * (z_jt_param / len(neighbors))

    #Freq dependent rule model Firth 2020
    elif NBDA_type == 3:
        #assert z_jt_type=="binary", "z_jt must be binary to appropriately calculate NBDA type 3 (conformity rule)"
        numerator = z_jt_param**f_SL
        denominator = z_jt_param**f_SL + ((len(neighbors)-z_jt_param) ** f_SL)
        transmission_func = s_param * (numerator/denominator)
        
    return transmission_func


The transmission function first makes a call to the `z_jt` function to get its value. The form is determined by the simulation variable `NBDA_z_jt_type`. The standard NBDA is `binary`, where a knowledgeable neighbor counts as +1. Setting this in FSSL decouples EWA from NBDA, and is included here for the purpose of comparison of simulations in which the two models are not allowed to inform each other. FSSL is meant to run with type `proportional`, in which information about how frequently the neighbor produced the behavior is included in their contribution to the value of $z_{jk}(t)$. This is done by dividing the number of behavioral productions of a given behavior by the total number of behaviors produced, such that a neighbors contribution can range from $[0,1]$.

In [ ]:
def z_jt(behavior, neighbors, G):
    z_jt = 0

    for neighbor in neighbors:
        if behavior in G.nodes[neighbor]["data"].knowledge.keys():
            #Values range integers between [0,length(neighbors)]
            if z_jt_type=="binary":
                z_jt += 1
            #Values range continuously from [0,length(neighbors)] with each neighbor weighted in proportion to solution type
            elif z_jt_type=="proportional":
            	#how many times a neighbor produces the behavior, over the total # of behaviors produced within the memory window
                z_jt += G.nodes[neighbor]["data"].ind_memory.count(behavior)/memory_window

    return z_jt

For future extensibility, the value of $A$ is defined as a function. This function could eventually make use of ILVs to calculate it's value. In this version, the function first checks the value of simulation variable `asocial_learning`. If this is false, $A=0$. If this is true and the NBDA is of the bounded form `NBDA_type=1`, $A=1-s$. Otherwise, $A=1$, which is now the accepted "default" in much of the literature.

In [ ]:
def A_param():
    if asocial_learning:
        if NBDA_type=="1":
            assert 0 <= s_param <=1, "s parameter must take a value between 0 and 1 for this type of NBDA."
            asocial_learning_func=1-s_param
        else:
            asocial_learning_func=1
    else:
        asocial_learning_func=0

    return asocial_learning_func

## Networks <a class="anchor" id="networks"></a>

Key to these simulations is that agents are situated within a social network. This is generated using the following function, which employs the `networkx` library. This is a powerful library with many pre-made network generation functions available. The full list at time of publication is [available here](https://networkx.org/documentation/stable/reference/generators.html). The type of graph is defined using the simulation parameter `graph_type`. We have included options for complete networks and random Erdős-Rényi graphs, and the import of a custom graph from an adjacency list. However, this function can be easily modified with other options.

Once the graph is created, each node is populated with an instance of the agent class as it's `data` key (each node is defined as a dictionary in networkx). Once an agent is created, a dice is rolled to see whether they will be initially knowledgable, governed by the simulation variable `intial_knowledgable_prop`. The agent is then preprogrammed with knowledge of behavior "a", an intial attraction score, and a memory of socially observing the behavior. How this is done is totally up to the design and purpose of the simulation.

In [ ]:
def generate_network(graph_type):
    if graph_type == "complete":
        G = nx.complete_graph(N)
    elif graph_type == "random_regular":
        G = nx.random_regular_graph(int(N/2), N, seed=None)
    elif graph_type == "random_erdos":
        G = nx.gnp_random_graph(N, 0.5333, seed=None, directed=False)
        while not nx.is_connected(G):
            G = nx.gnp_random_graph(N, 0.5333, seed=None, directed=False)
    elif graph_type == "random_barabasi":
        G = nx.barabasi_albert_graph(N, 4, seed=None)
    elif graph_type == "random_small_world":
        G = nx.connected_watts_strogatz_graph(N, int(N/2), 0, tries=200, seed=None)
    elif graph_type == "custom_adj_list":
        G = nx.read_adjlist(custom_adj_list_filename)
    else:
        print("Incorrect graph name!")

    random_agent = random.randint(0, N-1)
    for x in list(G.nodes()):
        G.nodes[x]['data'] = agent(x)
        if random.random() <= initial_knowledgable_prop:
            G.nodes[x]['data'].knowledge["a"] = {"a_mat": 0,"i_mat": 0,"s_mat":0,"p_mat":0}
            #G.nodes[x]['data'].A_mat_update("a")
            G.nodes[x]['data'].I_mat_update()
            G.nodes[x]['data'].S_mat_update()
            G.nodes[x]['data'].P_mat_update()
            G.nodes[x]['data'].naive=False
    return G

## Simulation <a class="anchor" id="simulation"></a>

We will now cover the structure of the simulation, and review the various helper functions it requires.

1. We assign the simulation a unique number
2. We create a csv to hold the data output
3. We generate a unique network for the simulation and populate it with agents
4. We begin a loop with one pass per timestep. In this loop:
    1. A counter object is created to keep track of the frequencies of each behavior
    2. A list of knowledgable agents is created. 
    3. Each knowledgable agent produces 1 behavior, which is added to the `beh_freqs` counter.
    4. After all knowledgable agents have produced their behavior for the timestep, every agent in the population 
        * consolidates their short term social memory into long term memory
        * prunes any individual memory that lays beyond the memory window
        * prunes any social observation memory that lays beyond the memory window
        * updates their social weight matrix and probability matrix
    6. The number knowledgable of the novel behavior is counted.
    7. All relevant data is written to the csv.
    8. Agents update their knowledge states using the NBDA equations. Here they may acquire a new behavior to produce in the next time step.
    9. Agents update their `exposure` variable, which is a count of how many time steps they have existed in the simulation
    10. If there is turnover and the timestep is one in which turnover occurs, the oldest agents are replaced with new agents.
5. When all timesteps have passed, the simulation ends.


In [ ]:
def simulation():
    global master_sim_num
    sim_num = master_sim_num
    master_sim_num += 1
    #create csv file with headers
    file_path = "../raw_data/_gtype_{}_sim_{}.csv".format(graph_type, sim_num)

    #if not isfile(file_path):
    create_csv(file_path)

    print("simulation {}".format(sim_num))

    np.random.seed() #reset seed for random num generation
    G = generate_network(graph_type) #create network populated with agents

    timestep=0
    while True:

        know_novel = [agent for agent in range(N) if "b" in G.nodes[agent]["data"].knowledge.keys()]
        num_know_novel = len(know_novel)

        if timestep%10==0:
            agent_matrix_values = peek_inside(G)

        #create counter object to keep track of frequencies of behaviors for this timestep
        beh_freqs = Counter()
        for behavior in all_behaviors.values():
            beh_freqs[behavior.name] += 0

        #create list of all knowledgable agents
        knowledgable = [agent for agent in range(N) if G.nodes[agent]["data"].naive == False ]

        #knowledgable individuals produce given number of behaviors beh_per_TS
        for individual in knowledgable:
            for interactions in range(beh_per_TS):
                produced_behavior = production_event(G, individual)
                beh_freqs[produced_behavior] += 1

        #resolves internal counts to the memory window
        for agent in range(N):
            G.nodes[agent]["data"].prune_ind_memory()
            G.nodes[agent]["data"].consolidate_social_memory()
            G.nodes[agent]["data"].prune_social_memory()
            G.nodes[agent]["data"].reset_temp_social_memory()
            if not G.nodes[agent]["data"].naive:
                G.nodes[agent]["data"].I_mat_update()
                G.nodes[agent]["data"].S_mat_update()
                G.nodes[agent]["data"].P_mat_update()

        # write data and end sim when at full diffusion
        if num_know_novel==N:
            write_csv(file_path,sim_num,timestep,num_know_novel,beh_freqs,agent_matrix_values)
            break

        #write data
        if timestep%10==0:
            write_csv(file_path,sim_num,timestep,num_know_novel,beh_freqs,agent_matrix_values)

        #run NBDA calculations
        NBDA(G)
        timestep+=1

### Helper functions <a class="anchor" id="helper-agent"></a>

`production_event` has the focal individual produce a behavior, and the focal individual's neighbors observe that behavioral production.

In [ ]:
def production_event(G, producer):
    beh_production = G.nodes[producer]["data"].produceBehavior()

    neighbors = [node for node in G.neighbors(producer)]
    for neighbor in neighbors:
        G.nodes[neighbor]["data"].observe(beh_production)
    return beh_production

`NBDA` simply loops through all agents and has them each call their `acquire_behavior` method.

In [ ]:
def NBDA(G):
    for agent in list(G.nodes()):
        G.nodes[agent]["data"].acquire_behavior(G)

### Helper functions: extracting data <a class="anchor" id="helper-extract"></a>

In [ ]:
def create_csv(file_path):
    #create variable names for behaviors to be included in simulation
    behavior_list = ["behavior_{}".format(behavior.name) for behavior in all_behaviors.values()]
    labels=["A_mat","I_mat","S_mat","P_mat"]
    agent_matrix_list = ["behavior_{}_{}".format(behavior.name,label) for behavior in all_behaviors.values() for label in labels]

    #writes header for main data
    with open(file_path,"w") as fout:
        fout.write("sim, graph_type, pop_size, memory_window, NBDA_type, NBDA_basehazard, NBDA_s_param, NBDA_z_jt_type, NBDA_conformity, EWA_soc_info_weight, EWA_recent_payoff_weight, EWA_conformity, EWA_tau, timestep, num_know_novel, {}, {}\n".format(",".join(behavior_list),",".join(agent_matrix_list)))


In [ ]:
def write_csv(file_path,sim_num,timestep,num_know_novel,beh_freqs,agent_matrix_values):
    behavior_counts = [str(beh) for beh in beh_freqs.values()]
    #print(behavior_counts)
    count_string = ",".join(behavior_counts)

    #writes header for main data
    with open(file_path,"a+") as fout:
        fout.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
        sim_num,
        graph_type,
        N,
        memory_window,
        NBDA_type,
        base_rate,
        s_param,
        z_jt_type,
        f_SL,
        sigma,
        phi,
        f_SL,
        tau,
        timestep,
        num_know_novel,
        count_string,
        agent_matrix_values))

In [ ]:
def agent_values(agent, behavior,G):
    #if the agent knows a behavior, return associated matrix values, otherwise return zeroes
    if behavior in G.nodes[agent]["data"].knowledge.keys():
        return [G.nodes[agent]["data"].knowledge[behavior]["a_mat"],
        np.exp(G.nodes[agent]["data"].knowledge[behavior]["i_mat"]),
        G.nodes[agent]["data"].knowledge[behavior]["s_mat"],
        G.nodes[agent]["data"].knowledge[behavior]["p_mat"]]
    else:
        return [0,0,0,0]

def peek_inside(G):
    to_write_list = []
    for behavior in all_behaviors.values():
        #values for A_mat, I_mat, S_mat and P_mat
        total_values = [0,0,0,0]
        for agent in range(N):
            agt_values = agent_values(agent,behavior.name,G)
            #print(agt_values)
            total_values = np.add(total_values,agt_values)
        to_write_list += list(total_values)

    agent_matrix_values_string = [str(v) for v in to_write_list]
    amv_string = ",".join(agent_matrix_values_string)
    return amv_string

## Running the simulation <a class="anchor" id="running"></a>

Finally, it's time to run the thing. This tutorial notebook has implemented a simpler version that runs one point in parameter space at a time, and doesn't capture full information from the agent's matrices. The full python version uses multi-threading to run these simulations much faster, across many points in parameter space.

### Simulation parameters <a class="anchor" id="sim_params"></a>
* `replicates` - number of replicates per point in parameter space
* `beh_per_TS` - number of behaviors individuals perform per timestep
* `master_sim_num` - begins at 0, this is incremented each simulation to give a unique sim ID
* `directory_path` - where raw data will be output to
* `new_directory_path` - where concatenated data will be output to at the end of all simulations

In [ ]:
replicates=3
master_sim_num=0
directory_path="../raw_data"
new_directory_path="../concat_data/csvs"
behavior_names = ["a","b"]
behavior_payoffs = [1,1]
beh_per_TS = 1 

### Population parameters <a class="anchor" id="pop_params"></a>

* `N` - population size
* `memory_window` - how far back the agent can remember (in time steps) the behaviors that other agents produce
* `initial_knowledgable_prop` - proportion of population that's initially knowledgable
* `graph_type` - type of network (random, complete, custom)
* `custom_adj_list_filename` - file path for custom network from adjacency list
* `turnover` - boolean indicating whether turnover occurs
* `turnover_interval` - interval in timesteps between turnover events
* `num_turnover` - how many agents should be replaced each turnover

In [ ]:
N = 16 #population size
initial_knowledgable_prop = .5 #initial proportion of knowledgable individuals in the population
full_initial_weights = False
graph_type = "random_regular" #"random_regular", "random_erdos","random_barabasi","random_small_world"
custom_adj_list_filename = ""

### EWA parameters <a class="anchor" id="EWA_params"></a>

* `sigma` - social information bias
* `phi` - recent payoff bias
* `f_SI` - conformity exponent
* `tau` - sensitivity to differences in attraction score, also called inverse exploration parameter
* `memory_window` - how far back agents can remember in timesteps

In [ ]:
#EWA Parameters
sigma = 0.5 
phi = 0.5 
f_SI = 1 
tau = 1 
memory_window = 25


### NBDA parameters <a class="anchor" id="NBDA_params"></a>

* `NBDA_type` - the form of NBDA, type 0 is the general unbounded form; 1: bounded, linked S and (1-S); 2: Proportional rule (Firth et al. 2020); 3: Conformity rule (Firth et al. 2020)
* `f_SL` - if using NBDA_type 3, the conformity exponent used by NBDA
* `s_param` - strength of social learning per unit of connection
* `z_jt_type` - #"binary" or "proportional", FSSL requires proportional
* `asocial_learning` - boolean value, True if asocial learning occurs
* `base_rate` - the base hazard is the underlying rate of transmission in every timestep

In [ ]:
NBDA_type = 0 # 0: Unbounded general form; 
f_SL = 1
s_param = 50 #s parameter from NBDA indicating strength of social learning per unit of connection
z_jt_type = "proportional" #"binary" or "proportional", proportional assigns z_j(t) a number between [0,1] depending on how frequently the produced behavior in previous timestep
asocial_learning = True #True or false, depending on if asocial learning occurs
base_rate = .01 #the base hazard is the underlying rate in every timestep that the behavior could be acquired

In [ ]:
#create dictionary of all behaviors
all_behaviors = dict((name, behavior(name,behavior_payoffs[i], base_rate)) for i,name in enumerate(behavior_names))

### Run the sim <a class="anchor" id="run_sim"></a>

Now that all of that is defined, we can run the simulation however many `replicates` we want. In the full version, each replicate is run in parallel using different threads.

In [ ]:
for x in range(replicates):
    simulation()

### Data management <a class="anchor" id="data_mgmt"></a>

Finally, we concatenate all the raw CSV outputs into one managable CSV file, and delete all of the raw data. Don't forget to change the name of the CSV the next time you run this! Otherwise the old data will be over-written.

In [ ]:
df_list = [pd.read_csv(join(directory_path,f)) for f in listdir(directory_path) if ".csv" in f]
df_concat = pd.concat(df_list)
df_concat.to_csv(join(new_directory_path,"concatenated_data.csv"), index = False)
for f in listdir(directory_path):
    if ".csv" in f:
        remove(join(directory_path,f))